In [7]:
import sys
sys.path.append(str('/home/jupyter/'))
import pandas as pd
from configparser import ConfigParser
from repo.MyConfigParser import MyConfigParser
from repo.LoadData import DataLoader
from repo.BuildModel import GenerateModelFile, CreateModel
from repo.Accuracy import AccuracyCalculator, ProphetParameterSearch, PlotAccuracyCharts, MakeFuture
from repo.FileGenerator import *
from repo.Optimization import SavantPSO

In [8]:
updated_config = ConfigParser()
config_in_path = "/home/jupyter/eu/24 Feb Update/config ups eu.ini"
updated_config.read(config_in_path)

getdict = MyConfigParser()
config_dict=getdict.GetDict(updated_config)
region = config_dict['CLIENTINFO']['country']
cutoff_month = config_dict['PROPHETSETTING']['cut_off_point']
base_start = config_dict['PROPHETSETTING']['reference_start_date1']
base_end = config_dict['PROPHETSETTING']['reference_end_date1']
periods = config_dict['PROPHETSETTING']['periods']
base_periods  = config_dict['PROPHETSETTING']['base_periods']

output_bucket = config_dict["FILEPATH"]["output_bucket"]
folder_name = config_dict["FILEPATH"]["folder_name"]

base_path = config_dict["FILEPATH"]["base_path"]

boundary_df_in_path = base_path + config_dict["FILEPATH"]["boundary_df_in_path"]
daily_df_out_path = base_path + config_dict["FILEPATH"]["daily_df_out_path"]
analytical_file_path = base_path + config_dict["FILEPATH"]["analytical_file_path"]
cuts_df_in_path = base_path + config_dict["FILEPATH"]["cuts_df_in_path"]
plot_name = base_path + config_dict["FILEPATH"]["plot_name"]
constraint_file_name = base_path + config_dict["FILEPATH"]["constraint_file_name"]
breakdown_ls = [config_dict['CLIENTINFO'][i] for i in [f'breakdown{i}' for i in range(1,5)] if config_dict['CLIENTINFO'][i] != None]

small_market_str = 'India|Saudi Arabia|UAE|Turkey|Ireland|Sweden|Netherlands|Belgium|Spain|Czech'

pre_forecast_period_budget = float(config_dict['CLIENTINFO']['pre_forecast_period_budget'])
client_budget = float(config_dict['CLIENTINFO']['client_budget'])

save_date = config_dict['CLIENTINFO']['save_date']

In [9]:
# Get forecasters
bdDf = pd.read_csv(cuts_df_in_path)#.drop('Unnamed: 0', axis = 1)
cuts_df = bdDf.reset_index(drop = True)
cuts_df['forecaster_name_list'] =  cuts_df['market'] + ' ' + cuts_df['account_type'] + ' ' + cuts_df['funnel'] + '.pkl' #' 05.19.2021 Final.pkl'
cuts_df['periods'] = periods
forecasters_array = cuts_df.forecaster_name_list

# Get analytical file
analytical_file = pd.read_csv(analytical_file_path).drop('Unnamed: 0', axis = 1)
analytical_file['ds'] = pd.to_datetime(analytical_file['ds'])
analytical_file_check_version = analytical_file.copy()

# Get constrain file
constraint_file = pd.read_csv(constraint_file_name)

In [10]:
today = '02.06.2024' #datetime.date.today().strftime("%m.%d.%Y")# 
fileName = base_path + f'Pos Result {today}.csv' #today
pos_df = pd.read_csv(fileName).drop('Unnamed: 0', axis = 1)
# pos_df = pd.DataFrame({'variable_splits':analytical_file.columns[19:37],'pos': 1})
# pos_df.pos = 1
pos_df

,variable_splits,pos
0,France_-_Brand_Jan_2024,1.083052
1,France_-_Brand_Feb_2024,1.026290
2,France_-_Brand_Mar_2024,0.897050
3,France_-_Brand_Apr_2024,0.976561
4,France_-_Brand_May_2024,1.014011
...,...,...
127,UK_-_Non_Brand_Aug_2024,0.960357
128,UK_-_Non_Brand_Sep_2024,0.877310
129,UK_-_Non_Brand_Oct_2024,0.963924
130,UK_-_Non_Brand_Nov_2024,1.092226


In [11]:
resultPSO = SavantPSO(cuts_df, analytical_file, constraint_file, config_dict)
inputWeights = pos_df.pos.to_numpy()

daily_simulated_forecast = resultPSO.ResultGenerator(inputWeights)

--------------Optimized spend isn't the same as what UPS planned.---------------
Test 1: total X next year:  6242581.0 difference between planned and forecast X:  0.0
yhat: 1413124.0
analytical_spend_latest_period: 5779500.0
client_budget - analytical_spend_latest_period: 0


In [12]:
# %load_ext autoreload
# %autoreload 2
resultPSO = SavantPSO(cuts_df, analytical_file, constraint_file, config_dict)
inputWeights_base = np.ones(constraint_file.shape[0])

daily_simulated_forecast_base = resultPSO.ResultGenerator(inputWeights_base)

--------------Optimized spend isn't the same as what UPS planned.---------------
Test 1: total X next year:  6242581.0 difference between planned and forecast X:  -0.0
yhat: 1387507.0
analytical_spend_latest_period: 5779500.0
client_budget - analytical_spend_latest_period: 0


In [13]:
forecasted_daily_df = daily_simulated_forecast[['market', 'account_type', 'funnel', 'ds', 'yhat', 'X']].copy()
forecasted_daily_df['month'] = pd.to_datetime(forecasted_daily_df['ds']).dt.strftime('%b') #+ '/' + pd.to_datetime(result_df['ds']).dt.strftime('%Y')
forecasted_daily_df['year'] = pd.to_datetime(forecasted_daily_df['ds']).dt.strftime('%Y').astype(str)

forecasted_daily_df = forecasted_daily_df.rename(columns = { 'yhat': 'OptimalConversionQuantity', 'X': 'OptimalSpendValue'})
forecasted_daily_df['OptimalSpendValue'] = forecasted_daily_df['OptimalSpendValue'].round(2)

forecasted_daily_df['variable_splits'] = forecasted_daily_df['market'] + '_' + forecasted_daily_df['account_type']+'_'+forecasted_daily_df['funnel']+'_'+forecasted_daily_df['month']+'_'+forecasted_daily_df['year']
print(forecasted_daily_df.OptimalSpendValue.sum(), forecasted_daily_df.OptimalConversionQuantity.sum())
forecasted_daily_df

6242581.42 1413124.0


,market,account_type,funnel,ds,OptimalConversionQuantity,OptimalSpendValue,month,year,variable_splits
0,France,-,Brand,2024-01-01,399.0,26.18,Jan,2024,France_-_Brand_Jan_2024
1,France,-,Non_Brand,2024-01-01,27.0,88.58,Jan,2024,France_-_Non_Brand_Jan_2024
2,Germany,-,Brand,2024-01-01,385.0,2025.81,Jan,2024,Germany_-_Brand_Jan_2024
3,Germany,-,Non_Brand,2024-01-01,188.0,8350.20,Jan,2024,Germany_-_Non_Brand_Jan_2024
4,Italy,-,Brand,2024-01-01,283.0,49.42,Jan,2024,Italy_-_Brand_Jan_2024
...,...,...,...,...,...,...,...,...,...
4021,Poland,-,Brand,2024-12-31,349.0,299.22,Dec,2024,Poland_-_Brand_Dec_2024
4022,Poland,-,Non_Brand,2024-12-31,0.0,32.13,Dec,2024,Poland_-_Non_Brand_Dec_2024
4023,SM,-,-,2024-12-31,812.0,4805.23,Dec,2024,SM_-_-_Dec_2024
4024,UK,-,Brand,2024-12-31,0.0,3427.68,Dec,2024,UK_-_Brand_Dec_2024


In [14]:
base_daily_df = daily_simulated_forecast_base[['market', 'account_type', 'funnel', 'ds', 'yhat', 'X']].copy()
base_daily_df['month'] = pd.to_datetime(base_daily_df['ds']).dt.strftime('%b') #+ '/' + pd.to_datetime(result_df['ds']).dt.strftime('%Y')
base_daily_df['year'] = pd.to_datetime(base_daily_df['ds']).dt.strftime('%Y').astype(str)
base_daily_df['X'] = base_daily_df['X'].round(2)
base_daily_df = base_daily_df.rename(columns = { 'X': 'BaseSpend', 'yhat': 'BaseConversionQuantity'})

print(base_daily_df.BaseSpend.sum(), base_daily_df.BaseConversionQuantity.sum())
base_daily_df

6242581.140000001 1387507.0


,market,account_type,funnel,ds,BaseConversionQuantity,BaseSpend,month,year
0,France,-,Brand,2024-01-01,399.0,24.51,Jan,2024
1,France,-,Non_Brand,2024-01-01,27.0,96.48,Jan,2024
2,Germany,-,Brand,2024-01-01,389.0,2057.06,Jan,2024
3,Germany,-,Non_Brand,2024-01-01,181.0,7557.30,Jan,2024
4,Italy,-,Brand,2024-01-01,283.0,48.82,Jan,2024
...,...,...,...,...,...,...,...,...
4021,Poland,-,Brand,2024-12-31,337.0,279.16,Dec,2024
4022,Poland,-,Non_Brand,2024-12-31,0.0,32.71,Dec,2024
4023,SM,-,-,2024-12-31,712.0,4140.93,Dec,2024
4024,UK,-,Brand,2024-12-31,0.0,4035.04,Dec,2024


In [15]:
forecast_n_base_df = pd.merge(forecasted_daily_df, base_daily_df, how = 'left', on = ['market', 'account_type', 'funnel', 'ds', 'month', 'year'])
forecast_n_base_df['SavedDate'] = datetime.datetime.strptime(save_date, '_%m_%d_%y').strftime("%Y-%m-%d")
forecast_n_base_df.tail()

,market,account_type,funnel,ds,OptimalConversionQuantity,OptimalSpendValue,month,year,variable_splits,BaseConversionQuantity,BaseSpend,SavedDate
4021,Poland,-,Brand,2024-12-31,349.0,299.22,Dec,2024,Poland_-_Brand_Dec_2024,337.0,279.16,2024-02-05
4022,Poland,-,Non_Brand,2024-12-31,0.0,32.13,Dec,2024,Poland_-_Non_Brand_Dec_2024,0.0,32.71,2024-02-05
4023,SM,-,-,2024-12-31,812.0,4805.23,Dec,2024,SM_-_-_Dec_2024,712.0,4140.93,2024-02-05
4024,UK,-,Brand,2024-12-31,0.0,3427.68,Dec,2024,UK_-_Brand_Dec_2024,0.0,4035.04,2024-02-05
4025,UK,-,Non_Brand,2024-12-31,46.0,148.71,Dec,2024,UK_-_Non_Brand_Dec_2024,46.0,155.74,2024-02-05


# Small Market

In [16]:
# forecast_n_base_df[forecast_n_base_df.ds>="2024-02-01"].sum(numeric_only=1)
forecast_n_base_df[forecast_n_base_df.ds>="2024-02-01"].groupby('market').sum(numeric_only=1)#.loc['SM',:'OptimalSpendValue']

,OptimalConversionQuantity,OptimalSpendValue,BaseConversionQuantity,BaseSpend
market,,,,
France,186647.0,1039561.35,187655.0,1068181.09
Germany,330582.0,1331639.45,332705.0,1354104.64
Italy,153899.0,581082.27,151828.0,562152.57
Poland,143468.0,141676.16,141659.0,138739.81
SM,364538.0,1453841.28,344233.0,1318755.83
UK,129402.0,1231699.57,124297.0,1337565.79


In [17]:
small_market_str

'India|Saudi Arabia|UAE|Turkey|Ireland|Sweden|Netherlands|Belgium|Spain|Czech'

In [18]:
daily_df = GetBQData(config_dict["PROPHETSETTING"]["training_query"], config_dict["PROPHETSETTING"]["input"], config_dict["PROPHETSETTING"]["output"])

df = daily_df[daily_df['market'].str.contains(small_market_str)].copy()
df = df.groupby(['region', 'market', 'account_type', 'funnel', 'ds']).sum().reset_index()
df = df.rename(columns={'ds':'date', 'X':'cost', 'y':'conversion_quantity'})

date_index = pd.date_range(df.date.min(), df.date.max()) # why was it df.ds.min and max?
multiple_index = pd.Index(date_index)

df = df.set_index(['date'])

df = df.reset_index()
df = df.fillna(1)
df.shape

(55043, 7)

In [19]:
# Get regional funnel split
regional_total_df = df[(df['funnel'].str.contains('Brand|Generic|Competitor'))].copy()
# regional_total_df.date = pd.to_datetime(regional_total_df.date)
regional_total_df = regional_total_df.groupby(['date']).sum().reset_index()

regional_total_df.rename(columns = {'cost': 'total_cost', 'dashboard_revenue': 'total_revenue', 'conversion_quantity': 'total_conversions'}, inplace = True)
regional_total_df.tail()

,date,region,market,account_type,funnel,total_cost,total_conversions
2335,2024-01-31,EuropeEuropeEuropeEuropeEuropeEuropeEuropeEuro...,BelgiumBelgiumBelgiumBelgiumBelgiumBelgiumCzec...,Bing AdsBing AdsBing AdsGoogle AdWordsGoogle A...,BrandCompetitorGenericBrandCompetitorGenericBr...,5262.5500,1195.0
2336,2024-02-01,EuropeEuropeEuropeEuropeEuropeEuropeEuropeEuro...,BelgiumBelgiumBelgiumBelgiumBelgiumBelgiumCzec...,Bing AdsBing AdsBing AdsGoogle AdWordsGoogle A...,BrandCompetitorGenericBrandCompetitorGenericBr...,5410.7400,1279.0
2337,2024-02-02,EuropeEuropeEuropeEuropeEuropeEuropeEuropeEuro...,BelgiumBelgiumBelgiumBelgiumBelgiumCzechCzechC...,Bing AdsBing AdsGoogle AdWordsGoogle AdWordsGo...,BrandCompetitorBrandCompetitorGenericBrandComp...,4638.0202,974.0
2338,2024-02-03,EuropeEuropeEuropeEuropeEuropeEuropeEuropeEuro...,BelgiumBelgiumBelgiumBelgiumCzechCzechCzechInd...,Bing AdsBing AdsGoogle AdWordsGoogle AdWordsBi...,BrandCompetitorBrandGenericBrandBrandCompetito...,1223.2800,243.0
2339,2024-02-04,EuropeEuropeEuropeEuropeEuropeEuropeEuropeEuro...,BelgiumBelgiumBelgiumBelgiumCzechCzechCzechInd...,Bing AdsBing AdsGoogle AdWordsGoogle AdWordsBi...,BrandCompetitorBrandGenericBrandBrandCompetito...,1062.9200,136.0


In [20]:
Brand_df = df[(df['funnel'] == 'Brand')][['date', 'cost', 'conversion_quantity']].copy()
Brand_df = Brand_df.groupby(['date']).sum().reset_index()
Brand_df.rename(columns = {'cost': 'brand_cost', 'conversion_quantity': 'brand_conversion_quantity'}, inplace = True)

Generic_df = df[df['funnel'] == 'Generic'][['date', 'cost', 'conversion_quantity']].copy()
Generic_df = Generic_df.groupby(['date']).sum().reset_index()
Generic_df.rename(columns = {'cost': 'generic_cost', 'conversion_quantity': 'generic_conversion_quantity'}, inplace = True)

Competitor_df = df[df['funnel'] == 'Competitor'][['date', 'cost', 'conversion_quantity']].copy()
Competitor_df = Competitor_df.groupby(['date']).sum().reset_index()
Competitor_df.rename(columns = {'cost': 'competitor_cost', 'conversion_quantity': 'competitor_conversion_quantity'}, inplace = True)

regional_split = regional_total_df.merge(Brand_df, how = 'left').merge(Generic_df, how = 'left').merge(Competitor_df, how = 'left')

regional_split['brand_cost_split'] = regional_split['brand_cost']/regional_split['total_cost']
regional_split['brand_conv_split'] = regional_split['brand_conversion_quantity']/regional_split['total_conversions']

regional_split['generic_cost_split'] = regional_split['generic_cost']/regional_split['total_cost']
regional_split['generic_conv_split'] = regional_split['generic_conversion_quantity']/regional_split['total_conversions']

regional_split['competitor_cost_split'] = regional_split['competitor_cost']/regional_split['total_cost']
regional_split['competitor_conv_split'] = regional_split['competitor_conversion_quantity']/regional_split['total_conversions']
regional_split = regional_split.fillna(0.33)
regional_split = regional_split[['date', 'brand_cost_split', 'brand_conv_split', 'generic_cost_split', 'generic_conv_split', 'competitor_cost_split', 'competitor_conv_split']]
regional_split

,date,brand_cost_split,brand_conv_split,generic_cost_split,generic_conv_split,competitor_cost_split,competitor_conv_split
0,2017-08-10,0.170333,0.995575,0.829667,0.004425,0.330000,0.330000
1,2017-08-11,0.574047,0.993579,0.425953,0.006421,0.330000,0.330000
2,2017-08-12,0.330000,0.990698,0.330000,0.009302,0.330000,0.330000
3,2017-08-13,0.330000,0.953488,0.330000,0.046512,0.330000,0.330000
4,2017-08-14,0.577467,0.988701,0.422533,0.011299,0.330000,0.330000
...,...,...,...,...,...,...,...
2335,2024-01-31,0.909553,0.895397,0.031116,0.017573,0.059331,0.087029
2336,2024-02-01,0.924569,0.893667,0.028392,0.017201,0.047040,0.089132
2337,2024-02-02,0.932706,0.871663,0.018872,0.027721,0.048422,0.100616
2338,2024-02-03,0.927596,0.930041,0.014355,0.016461,0.058049,0.053498


In [21]:
regional_split.date = pd.to_datetime(regional_split.date)
temp1 = regional_split[(regional_split['date'] <= base_end) & (regional_split['date'] >= base_start)].copy()
temp1['date'] = temp1['date'] + datetime.timedelta( days= base_periods)

# temp2 = regional_split[(regional_split['date'] <= base_end2) & (regional_split['date'] >= base_start2)].copy()
# temp2['date'] = temp2['date'] + datetime.timedelta( days= periods2)

regional_split_upd = pd.concat([temp1]) #, temp2
regional_split_upd.rename(columns = {'date': 'ds'}, inplace = True)
regional_split_upd

,ds,brand_cost_split,brand_conv_split,generic_cost_split,generic_conv_split,competitor_cost_split,competitor_conv_split
1941,2024-01-01,0.499306,0.850348,0.119516,0.003480,0.381177,0.146172
1942,2024-01-02,0.503893,0.798173,0.097364,0.013289,0.398742,0.188538
1943,2024-01-03,0.524805,0.824881,0.144044,0.023772,0.331150,0.151347
1944,2024-01-04,0.516269,0.836237,0.173958,0.022648,0.309773,0.141115
1945,2024-01-05,0.677750,0.829787,0.137651,0.027996,0.184599,0.142217
...,...,...,...,...,...,...,...
2302,2024-12-27,0.669414,0.845455,0.228436,0.022727,0.102150,0.131818
2303,2024-12-28,0.760131,0.903030,0.094872,0.000000,0.144996,0.096970
2304,2024-12-29,0.790753,0.878788,0.000000,0.000000,0.209247,0.121212
2305,2024-12-30,0.679987,0.904762,0.143509,0.015873,0.176504,0.079365


In [22]:
idx = pd.date_range(regional_split_upd.ds.min(), regional_split_upd.ds.max())
regional_split_upd = regional_split_upd.set_index('ds')
regional_split_upd = regional_split_upd.reindex(idx).fillna(0.333).rename_axis('ds').reset_index()
regional_split_upd

,ds,brand_cost_split,brand_conv_split,generic_cost_split,generic_conv_split,competitor_cost_split,competitor_conv_split
0,2024-01-01,0.499306,0.850348,0.119516,0.003480,0.381177,0.146172
1,2024-01-02,0.503893,0.798173,0.097364,0.013289,0.398742,0.188538
2,2024-01-03,0.524805,0.824881,0.144044,0.023772,0.331150,0.151347
3,2024-01-04,0.516269,0.836237,0.173958,0.022648,0.309773,0.141115
4,2024-01-05,0.677750,0.829787,0.137651,0.027996,0.184599,0.142217
...,...,...,...,...,...,...,...
361,2024-12-27,0.669414,0.845455,0.228436,0.022727,0.102150,0.131818
362,2024-12-28,0.760131,0.903030,0.094872,0.000000,0.144996,0.096970
363,2024-12-29,0.790753,0.878788,0.000000,0.000000,0.209247,0.121212
364,2024-12-30,0.679987,0.904762,0.143509,0.015873,0.176504,0.079365


In [23]:
regional_split_upd[['brand_cost_split', 'generic_cost_split', 'competitor_cost_split']].sum(axis=1).unique()

array([1., 1., 1., 1.])

In [24]:
sm_df = forecast_n_base_df[forecast_n_base_df['market'] == 'SM'].copy()
sm_df = sm_df.merge(regional_split_upd, how = 'left', left_on = 'ds', right_on='ds')

brand_df = sm_df.copy()
brand_df['funnel'] = 'Brand'
brand_df['OptimalSpendValue'] = brand_df['OptimalSpendValue'] * brand_df['brand_cost_split']
brand_df['OptimalConversionQuantity'] = brand_df['OptimalConversionQuantity'] * brand_df['brand_conv_split']
brand_df['BaseSpend'] = brand_df['BaseSpend'] * brand_df['brand_cost_split']
brand_df['BaseConversionQuantity'] = brand_df['BaseConversionQuantity'] * brand_df['brand_conv_split']

generic_df = sm_df.copy()
generic_df['funnel'] = 'Generic'
generic_df['OptimalSpendValue'] = generic_df['OptimalSpendValue'] * generic_df['generic_cost_split']
generic_df['OptimalConversionQuantity'] = generic_df['OptimalConversionQuantity'] * generic_df['generic_conv_split']
generic_df['BaseSpend'] = generic_df['BaseSpend'] * generic_df['generic_cost_split']
generic_df['BaseConversionQuantity'] = generic_df['BaseConversionQuantity'] * generic_df['generic_conv_split']

competitor_df = sm_df.copy()
competitor_df['funnel'] = 'Competitor'
competitor_df['OptimalSpendValue'] = competitor_df['OptimalSpendValue'] * competitor_df['competitor_cost_split']
competitor_df['OptimalConversionQuantity'] = competitor_df['OptimalConversionQuantity'] * competitor_df['competitor_conv_split']
competitor_df['BaseSpend'] = competitor_df['BaseSpend'] * competitor_df['competitor_cost_split']
competitor_df['BaseConversionQuantity'] = competitor_df['BaseConversionQuantity'] * competitor_df['competitor_conv_split']

sm_funnel_df = pd.concat([brand_df, generic_df, competitor_df])
sm_funnel_df.sum(numeric_only=1)

OptimalConversionQuantity    3.923360e+05
OptimalSpendValue            1.544855e+06
BaseConversionQuantity       3.728260e+05
BaseSpend                    1.415057e+06
brand_cost_split             9.181973e+02
brand_conv_split             9.751421e+02
generic_cost_split           7.096108e+01
generic_conv_split           2.550051e+01
competitor_cost_split        1.088416e+02
competitor_conv_split        9.735735e+01
dtype: float64

In [25]:
sm_funnel_df[sm_funnel_df.ds>="2024-02-01"].groupby('market').sum(numeric_only=1).OptimalSpendValue[0] == forecast_n_base_df[forecast_n_base_df.ds>="2024-02-01"].groupby('market').sum(numeric_only=1).loc['SM','OptimalSpendValue']

True

In [26]:
# get small country split based on reference date range
small_country_split_df = df[df['market'].str.contains(small_market_str)&(df.date>=base_start)&(df.date<=base_end)].copy()
small_country_split_df.date = pd.to_datetime(small_country_split_df.date)
# small_country_split_df = small_country_split_df[(small_country_split_df['date'] >= cutoff_month.strftime('%Y-%m-%d'))]
# austria_switzl_df = df[(df.market.str.contains('Austria|Switzerland'))&(df.date>='2023-01-01')&(df.date<='2023-01-31')]
# austria_switzl_df.date = austria_switzl_df.date+datetime.timedelta(days=365)
# small_country_split_df = pd.concat([small_country_split_df, austria_switzl_df])
small_country_split_df = small_country_split_df.groupby(['market']).sum(numeric_only=1).reset_index()
small_country_split_df['cost_split'] = small_country_split_df['cost']/small_country_split_df['cost'].sum(numeric_only=1)
# small_country_split_df['revenue_split'] = small_country_split_df['dashboard_revenue']/small_country_split_df['dashboard_revenue'].sum(numeric_only=1)
small_country_split_df['conversion_split'] = small_country_split_df['conversion_quantity']/small_country_split_df['conversion_quantity'].sum(numeric_only=1)
small_country_split_df.fillna(0, inplace = True)
small_country_split_df
# austria_df

,market,cost,conversion_quantity,cost_split,conversion_split
0,Belgium,164939.8984,36964.0,0.112355,0.089954
1,Czech,47125.0878,10635.0,0.032101,0.025881
2,India,293470.9004,67601.0,0.199909,0.164511
3,Ireland,68094.6203,35206.0,0.046385,0.085676
4,Netherlands,316144.5797,93933.0,0.215354,0.228591
5,Saudi Arabia,33037.3799,9458.0,0.022505,0.023017
6,Spain,297881.9761,74929.0,0.202914,0.182344
7,Sweden,80542.9820,32056.0,0.054865,0.078010
8,Turkey,60804.9288,35065.0,0.041420,0.085332
9,UAE,105978.9131,15075.0,0.072192,0.036686


In [27]:
sm_fianl_df = pd.DataFrame()
for i in small_market_str.split('|'):
    sm_funnel_df_temp = sm_funnel_df.copy()
    sm_funnel_df_temp['market'] = i
    sm_funnel_df_temp['OptimalSpendValue'] = sm_funnel_df_temp['OptimalSpendValue'] * small_country_split_df.loc[small_country_split_df['market'] == i, 'cost_split'].array[0]
    sm_funnel_df_temp['OptimalConversionQuantity'] = sm_funnel_df_temp['OptimalConversionQuantity'] * small_country_split_df.loc[small_country_split_df['market'] == i, 'conversion_split'].array[0]
    sm_funnel_df_temp['BaseSpend'] = sm_funnel_df_temp['BaseSpend'] * small_country_split_df.loc[small_country_split_df['market'] == i, 'cost_split'].array[0]
    sm_funnel_df_temp['BaseConversionQuantity'] = sm_funnel_df_temp['BaseConversionQuantity'] * small_country_split_df.loc[small_country_split_df['market'] == i, 'conversion_split'].array[0]
    sm_fianl_df = pd.concat([sm_fianl_df, sm_funnel_df_temp])
    
sm_fianl_df = sm_fianl_df.drop(['variable_splits', 'generic_cost_split', 'competitor_cost_split', 'brand_cost_split', 'brand_conv_split', 'generic_conv_split', 'competitor_conv_split'], axis = 1)
sm_fianl_df = sm_fianl_df.rename(columns=lambda x: x[0:1].upper() + x[1:])
sm_fianl_df

,Market,Account_type,Funnel,Ds,OptimalConversionQuantity,OptimalSpendValue,Month,Year,BaseConversionQuantity,BaseSpend,SavedDate
0,India,-,Brand,2024-01-01,94.986134,222.237060,Jan,2024,97.644067,235.147246,2024-02-05
1,India,-,Brand,2024-01-02,113.975197,337.038193,Jan,2024,117.914432,356.618655,2024-02-05
2,India,-,Brand,2024-01-03,133.123311,437.487681,Jan,2024,138.008570,462.903993,2024-02-05
3,India,-,Brand,2024-01-04,126.976916,396.929794,Jan,2024,131.654288,419.990355,2024-02-05
4,India,-,Brand,2024-01-05,87.229089,340.931130,Jan,2024,90.232281,360.738197,2024-02-05
...,...,...,...,...,...,...,...,...,...,...,...
361,Czech,-,Competitor,2024-12-27,0.733486,4.114602,Dec,2024,0.644785,3.545774,2024-02-05
362,Czech,-,Competitor,2024-12-28,0.446719,4.170654,Dec,2024,0.401545,3.594050,2024-02-05
363,Czech,-,Competitor,2024-12-29,0.188224,2.422714,Dec,2024,0.163128,2.087733,2024-02-05
364,Czech,-,Competitor,2024-12-30,0.525833,6.957638,Dec,2024,0.474482,5.995784,2024-02-05


In [28]:
sm_fianl_df[sm_fianl_df.Ds>="2024-02-01"].sum(numeric_only=1).OptimalSpendValue == forecast_n_base_df[forecast_n_base_df.ds>="2024-02-01"].groupby('market').sum(numeric_only=1).loc['SM','OptimalSpendValue']

True

# Large Market

In [29]:
analytical_file[(analytical_file.generic_cost_split+analytical_file.competitor_cost_split > 1)]

,ds,region,market,account_type,funnel,Identifier,X,Nov_Dec_2018,Jul_Dec_2019,weekday_or_weekend,...,UK_-_Non_Brand_Dec_2024,UK_-_Non_Brand_Feb_2024,UK_-_Non_Brand_Jan_2024,UK_-_Non_Brand_Jul_2024,UK_-_Non_Brand_Jun_2024,UK_-_Non_Brand_Mar_2024,UK_-_Non_Brand_May_2024,UK_-_Non_Brand_Nov_2024,UK_-_Non_Brand_Oct_2024,UK_-_Non_Brand_Sep_2024
21,2024-01-02,Europe,UK,-,Non_Brand,L,217.8900,0.0,0.0,0.0,...,0.0,0.0,217.89,0.0,0.0,0.0,0.0,0.0000,0.0,0.00
478,2024-02-13,Europe,Italy,-,Non_Brand,L,209.0600,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0000,0.0,0.00
1033,2024-04-03,Europe,UK,-,Non_Brand,L,480.3100,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0000,0.0,0.00
1041,2024-04-04,Europe,Poland,-,Non_Brand,L,254.4900,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0000,0.0,0.00
1272,2024-04-25,Europe,Poland,-,Non_Brand,L,225.8794,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0000,0.0,0.00
1415,2024-05-08,Europe,Poland,-,Non_Brand,L,423.2600,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0000,0.0,0.00
2705,2024-09-02,Europe,UK,-,Non_Brand,L,475.5900,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0000,0.0,475.59
3398,2024-11-04,Europe,UK,-,Non_Brand,L,924.5177,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,924.5177,0.0,0.00


In [30]:
lm_df = forecast_n_base_df[forecast_n_base_df['market'] != 'SM'].copy()
lm_split_df = pd.merge(lm_df, analytical_file[['market', 'funnel', 'ds', 'generic_cost_split', 'competitor_cost_split']], how = 'left', on = ['market', 'funnel', 'ds'])

# helper_df is used to get the investment split for each funnel
lm_generic_helper_df = lm_split_df.loc[lm_split_df['funnel'] == 'Non_Brand', ].copy()
lm_generic_helper_df.loc[:,'funnel'] = 'Generic'
lm_generic_helper_df.loc[:,'OptimalSpendValue'] = lm_generic_helper_df['OptimalSpendValue'] * lm_generic_helper_df['generic_cost_split']
lm_generic_helper_df.loc[:,'OptimalConversionQuantity'] = lm_generic_helper_df['OptimalConversionQuantity'] * lm_generic_helper_df['generic_cost_split']#round(lm_generic_helper_df['OptimalConversionQuantity'] * lm_generic_helper_df['generic_cost_split'], 0)
lm_generic_helper_df.loc[:,'BaseSpend'] = lm_generic_helper_df['BaseSpend'] * lm_generic_helper_df['generic_cost_split']
lm_generic_helper_df.loc[:,'BaseConversionQuantity'] = lm_generic_helper_df['BaseConversionQuantity'] * lm_generic_helper_df['generic_cost_split'] #round(lm_generic_helper_df['BaseConversionQuantity'] * lm_generic_helper_df['generic_cost_split'], 0)

lm_competitor_helper_df = lm_split_df.loc[lm_split_df['funnel'] == 'Non_Brand', ].copy()
lm_competitor_helper_df.loc[:,'funnel'] = 'Competitor'
lm_competitor_helper_df.loc[:,'OptimalSpendValue'] = lm_competitor_helper_df['OptimalSpendValue'] * lm_competitor_helper_df['competitor_cost_split']
lm_competitor_helper_df.loc[:,'OptimalConversionQuantity'] = lm_competitor_helper_df['OptimalConversionQuantity'] * lm_competitor_helper_df['competitor_cost_split'] #round(lm_competitor_helper_df['OptimalConversionQuantity'] * lm_competitor_helper_df['competitor_cost_split'], 0)
lm_competitor_helper_df.loc[:,'BaseSpend'] = lm_competitor_helper_df['BaseSpend'] * lm_competitor_helper_df['competitor_cost_split']
lm_competitor_helper_df.loc[:,'BaseConversionQuantity'] = lm_competitor_helper_df['BaseConversionQuantity'] * lm_competitor_helper_df['competitor_cost_split'] #round(lm_competitor_helper_df['BaseConversionQuantity'] * lm_competitor_helper_df['competitor_cost_split'], 0)

lm_funnel_level_daily_df = pd.concat([lm_split_df[(lm_split_df['funnel'] == 'Brand') | (lm_split_df['funnel'] == '-')], lm_generic_helper_df, lm_competitor_helper_df])
print(lm_funnel_level_daily_df.market.unique(), lm_funnel_level_daily_df.sum(numeric_only=1).OptimalSpendValue)
lm_funnel_level_daily_df

['France' 'Germany' 'Italy' 'Poland' 'UK'] 4697726.84


,market,account_type,funnel,ds,OptimalConversionQuantity,OptimalSpendValue,month,year,variable_splits,BaseConversionQuantity,BaseSpend,SavedDate,generic_cost_split,competitor_cost_split
0,France,-,Brand,2024-01-01,399.000000,26.180000,Jan,2024,France_-_Brand_Jan_2024,399.000000,24.510000,2024-02-05,0.500000,0.500000
2,Germany,-,Brand,2024-01-01,385.000000,2025.810000,Jan,2024,Germany_-_Brand_Jan_2024,389.000000,2057.060000,2024-02-05,0.500000,0.500000
4,Italy,-,Brand,2024-01-01,283.000000,49.420000,Jan,2024,Italy_-_Brand_Jan_2024,283.000000,48.820000,2024-02-05,0.500000,0.500000
6,Poland,-,Brand,2024-01-01,316.000000,126.280000,Jan,2024,Poland_-_Brand_Jan_2024,310.000000,116.990000,2024-02-05,0.500000,0.500000
8,UK,-,Brand,2024-01-01,235.000000,17.890000,Jan,2024,UK_-_Brand_Jan_2024,235.000000,18.260000,2024-02-05,0.500000,0.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3651,France,-,Competitor,2024-12-31,0.774517,2.164452,Dec,2024,France_-_Non_Brand_Dec_2024,0.774517,2.257072,2024-02-05,0.967728,0.032272
3653,Germany,-,Competitor,2024-12-31,36.318566,151.355514,Dec,2024,Germany_-_Non_Brand_Dec_2024,36.318566,161.735867,2024-02-05,0.155382,0.844618
3655,Italy,-,Competitor,2024-12-31,6.201084,153.593094,Dec,2024,Italy_-_Non_Brand_Dec_2024,6.201084,166.940927,2024-02-05,0.224865,0.775135
3657,Poland,-,Competitor,2024-12-31,0.000000,19.319690,Dec,2024,Poland_-_Non_Brand_Dec_2024,0.000000,19.668443,2024-02-05,0.398702,0.601298


In [31]:
lm_funnel_level_daily_df = lm_funnel_level_daily_df.rename(columns=lambda x: x[0:1].upper() + x[1:])
lm_funnel_level_daily_df = lm_funnel_level_daily_df.drop(['Variable_splits', 'Generic_cost_split', 'Competitor_cost_split'], axis = 1)
lm_funnel_level_daily_df.tail()

,Market,Account_type,Funnel,Ds,OptimalConversionQuantity,OptimalSpendValue,Month,Year,BaseConversionQuantity,BaseSpend,SavedDate
3651,France,-,Competitor,2024-12-31,0.774517,2.164452,Dec,2024,0.774517,2.257072,2024-02-05
3653,Germany,-,Competitor,2024-12-31,36.318566,151.355514,Dec,2024,36.318566,161.735867,2024-02-05
3655,Italy,-,Competitor,2024-12-31,6.201084,153.593094,Dec,2024,6.201084,166.940927,2024-02-05
3657,Poland,-,Competitor,2024-12-31,0.000000,19.319690,Dec,2024,0.000000,19.668443,2024-02-05
3659,UK,-,Competitor,2024-12-31,3.458589,11.181016,Dec,2024,3.458589,11.709579,2024-02-05


In [32]:
# lm_funnel_level_daily_df[lm_funnel_level_daily_df.Ds>="2024-02-01"].groupby('Market').sum(numeric_only=1)
lm_funnel_level_daily_df[lm_funnel_level_daily_df.Ds>="2024-02-01"].sum(numeric_only=1)

OptimalConversionQuantity     943998.0
OptimalSpendValue            4325658.8
BaseConversionQuantity        938144.0
BaseSpend                    4460743.9
dtype: float64

In [33]:
# lm_funnel_level_daily_df[lm_funnel_level_daily_df.Ds>="2024-02-01"].sum(numeric_only=1).OptimalSpendValue == forecast_n_base_df[(forecast_n_base_df.ds>="2024-02-01")|(for]ecast_n_base_df.market!="SM")].sum(numeric_only=1).OptimalSpendValue
print(lm_funnel_level_daily_df[lm_funnel_level_daily_df.Ds>="2024-02-01"].sum(numeric_only=1).OptimalSpendValue, forecast_n_base_df[(forecast_n_base_df.ds>="2024-02-01")&(forecast_n_base_df.market!="SM")].sum(numeric_only=1).OptimalSpendValue)

4325658.8 4325658.8


In [34]:
LM_SM_three_funnel_df = pd.concat([sm_fianl_df, lm_funnel_level_daily_df]).reset_index(drop = True)
LM_SM_three_funnel_df.insert(3, 'WeekStart', [ i for i in LM_SM_three_funnel_df.Ds.apply(lambda x: x - pd.offsets.Week(weekday=4) )])
LM_SM_three_funnel_df.insert(0, 'Region', 'EMEA')
LM_SM_three_funnel_df.tail()

,Region,Market,Account_type,Funnel,WeekStart,Ds,OptimalConversionQuantity,OptimalSpendValue,Month,Year,BaseConversionQuantity,BaseSpend,SavedDate
16465,EMEA,France,-,Competitor,2024-12-27,2024-12-31,0.774517,2.164452,Dec,2024,0.774517,2.257072,2024-02-05
16466,EMEA,Germany,-,Competitor,2024-12-27,2024-12-31,36.318566,151.355514,Dec,2024,36.318566,161.735867,2024-02-05
16467,EMEA,Italy,-,Competitor,2024-12-27,2024-12-31,6.201084,153.593094,Dec,2024,6.201084,166.940927,2024-02-05
16468,EMEA,Poland,-,Competitor,2024-12-27,2024-12-31,0.000000,19.319690,Dec,2024,0.000000,19.668443,2024-02-05
16469,EMEA,UK,-,Competitor,2024-12-27,2024-12-31,3.458589,11.181016,Dec,2024,3.458589,11.709579,2024-02-05


In [36]:
LM_SM_three_funnel_df[LM_SM_three_funnel_df.Ds>='2024-02-01'].sum(numeric_only=1)

OptimalConversionQuantity    1308536.00
OptimalSpendValue            5779500.08
BaseConversionQuantity       1282377.00
BaseSpend                    5779499.73
dtype: float64

# Save to BQ

In [37]:
# Construct a BigQuery client object.
bqclient = bigquery.Client()

target_table_name = 'budget_allocation_scenario_with_base_' + region + save_date
table_id = "ups-analytics.forecast_optimization." + target_table_name
print(table_id)

ups-analytics.forecast_optimization.budget_allocation_scenario_with_base_Europe_02_05_24


In [38]:
# job_config.
job_config = bigquery.LoadJobConfig(write_disposition = "WRITE_TRUNCATE")

load_job = bqclient.load_table_from_dataframe(
    LM_SM_three_funnel_df, table_id, job_config=job_config
)  # Make an API request.

load_job.result()  # Waits for the job to complete.

destination_table = bqclient.get_table(table_id)  # Make an API request.
print("{} loaded {} rows.".format(target_table_name, destination_table.num_rows))

budget_allocation_scenario_with_base_Europe_02_05_24 loaded 16470 rows.


table = bqclient.get_table(table_id)  # Make an API request.
print(
    "Loaded {} rows and {} columns to {}".format(
        table.num_rows, len(table.schema), table_id
    )
)

In [39]:
LM_SM_three_funnel_df[LM_SM_three_funnel_df.Ds>='2024-01-01'].to_csv(base_path + 'output.csv')

In [33]:
LM_SM_three_funnel_df[LM_SM_three_funnel_df.Ds>='2024-02-01'].sum(numeric_only=1)

OptimalConversionQuantity    1668099.00
OptimalSpendValue            5779500.16
BaseConversionQuantity       1667902.00
BaseSpend                    5779499.73
dtype: float64